In [1]:
import pandas as pd
import datetime
from collections import defaultdict
import numpy as np
import itertools

In [2]:
import pymysql

# DB Info
HOST = "127.0.0.1"
PORT = 3306
DATABASE = "SMARTSYSTEM"
USER = "LSIS"
PWD = "LSISno1!"

class DB:
    def __init__(self, host, port, db, user, pwd) -> None:
        self.conn = None
        self.cursor = None
        self.init(host, port, db, user, pwd)

    def __enter__(self):
        return self

    def execute(self, query):
        self.cursor.execute(query)

    def commit(self):
        self.conn.commit()

    def fetchall(self):
        return self.cursor.fetchall()

    def init(self, host, port, db, user, pwd):
        self.conn = pymysql.connect(
            host=host, db=db, user=user, password=pwd, port=port, charset="utf8")
        self.cursor = self.conn.cursor()

    def close(self):
        self.conn.close()
        self.cursor = None

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()


def query_get_id(browsepath):
    return f'SELECT id FROM namekeyinfo WHERE browsePath="{browsepath}";'

def _convert_daterange(start, end=None):
    if end is None:
        start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
        end_date = start_date + datetime.timedelta(days=1)
    else:
        end_date = datetime.datetime.strptime(end, "%Y-%m-%d")
        end_date = end_date + datetime.timedelta(days=1)
        
    end_date = end_date.strftime("%Y-%m-%d")
        
    return start, end_date

## 3상 THD
def query_get_thdval(phase, id, start_date, end_date=None):
    phase = phase.lower()

    start, end = _convert_daterange(start_date, end_date)    

    return f'SELECT timestamp, value, `minValue`, `maxValue`, "{phase.upper()}" as `phase` FROM analyticsthdi{phase}1min WHERE id="{id}" AND timestamp >= "{start}" AND timestamp < "{end}";'

## 3상 TDD
def query_get_tddval(phase, id, start_date, end_date=None):
    phase = phase.lower()

    start, end = _convert_daterange(start_date, end_date)    

    return f'SELECT timestamp, value, `minValue`, `maxValue`, "{phase.upper()}" as `phase` FROM analyticstddi{phase}1min WHERE id="{id}" AND timestamp >= "{start}" AND timestamp < "{end}";'


In [3]:
def fit_timeindex(df):
    """
    A, B, C 모두 동일한 분단위 시간 축으로 변환
    """
    if "phase" in df.columns:
        # 중복시간 제거
        df_tmp = df.drop_duplicates(subset=["phase", "timestamp"], keep="first")

        xmin = df_tmp["timestamp"].min().replace(hour=0, minute=0)
        xmax = df_tmp["timestamp"].max().replace(hour=23, minute=59)

        # pd.date_range,explode 사용해서 빠진 시간 정보 채워 넣고, 만들어진 시간 정보에 원보 데이터 밀어 넣고 공백은 NaN으로
        df_tmp = (
            df_tmp.groupby(["phase"])["timestamp"]
            .apply(lambda x: pd.date_range(start=xmin, end=xmax, freq="min"))
            .explode()
            .reset_index()
            .merge(df_tmp, how="left", on=["phase", "timestamp"])
            .fillna(np.NaN)
        )
    else:
        # 중복시간 제거
        df_tmp = df.drop_duplicates(subset=["timestamp"], keep="first")

        xmin = df_tmp["timestamp"].min().replace(hour=0, minute=0)
        xmax = df_tmp["timestamp"].max().replace(hour=23, minute=59)

        # pd.date_range,explode 사용해서 빠진 시간 정보 채워 넣고, 만들어진 시간 정보에 원보 데이터 밀어 넣고 공백은 NaN으로
        df_datetime = pd.DataFrame(index=pd.date_range(start=xmin, end=xmax, freq="min", name="timestamp")).reset_index()
        df_tmp = pd.merge(df_datetime, df_tmp, how="left", on=["timestamp"]).fillna(np.NaN)
    return df_tmp

In [4]:


def get_tags():
    df_namekey = pd.read_excel("./data/thdia_namekeyinfo.xlsx", "Sheet1")
    df_tag = df_namekey[["browsePath", "id"]]
    return df_tag.values.tolist()

def get_tablenames():
    df_tablenames = pd.read_csv("./data/tablename.csv")
    df_tablenames = df_tablenames.dropna(how="any")
    df_tablenames = df_tablenames.reset_index(drop=True)
    return df_tablenames

def get_date(date:str="2022-01-27"):
    return datetime.datetime.strptime(date, "%Y-%m-%d").date()

def get_dtype(columns: list):
    dict_dtype = defaultdict(lambda: "float64")
    dict_dtype["timestamp"] = "datetime64[s]"
    
    for column in columns:
        if column == "phase":
            dict_dtype[column] = "string"
        # print(dict_dtype[column])
    return dict(dict_dtype)

def get_columns(add_columns :list = []):
    return ["timestamp", "value", "minValue", "maxValue"] + add_columns


def get_data_all_phase(index, start_date, end_date=None):
    browsepath, id = get_tags()[index]    
    print(f"browsepath = {browsepath}")
    print(f"id = {id}")
    
    columns = get_columns(["phase"])
    dtypes = get_dtype(columns)
    
    with DB(HOST, PORT, DATABASE, USER, PWD) as db:
        query_list = []
        # functions = (query_get_thdval,query_get_tddval)
        functions = (query_get_thdval,)
        results = [pd.DataFrame(columns=columns) for _ in range(len(functions))]

        for phase in ["A", "B", "C"]:
            for idx, func in enumerate(functions):
                query = func(phase, id, start_date, end_date)
                query_list.append(query)

                db.execute(query)
                rows = db.fetchall()

                df_tmp = pd.DataFrame(rows, columns=columns)
                results[idx] = pd.concat([results[idx], df_tmp])

        results = [fit_timeindex(df_tmp.reset_index(drop=True).astype(dtypes)) for df_tmp in results]

        # *others, l4, l5, l6, l7 = browsepath.split(".")

        # result = results[0].copy()
        # result['l4'] = l4
        # result['l5'] = l5
        # result['l6'] = l6
        # result['l7'] = l7
    return results, query_list, browsepath

In [52]:
start_date = "2022-02-10"
end_date = "2022-02-12"

from collections import defaultdict

class DATA:
    def __init__(self) -> None:
        self.data = defaultdict(list)

    def add(self, browsepath, results):
        self.data[browsepath].append(results)
        

data = DATA()
for order in range(0,2):
    results, query_tmp, browsepath = get_data_all_phase(order, start_date, end_date)
    for result in results:
        data.add(browsepath, result)


browsepath = SmartMV.LSM_JH.FEMS.1ST_ELEC.LV.ACB_1.ACBOCR_S
id = 3:452092
browsepath = SmartMV.LSM_JH.FEMS.1ST_ELEC.LV.ACB_10.ACBOCR_S
id = 3:467156


In [55]:
data.data.keys()

dict_keys(['SmartMV.LSM_JH.FEMS.1ST_ELEC.LV.ACB_1.ACBOCR_S', 'SmartMV.LSM_JH.FEMS.1ST_ELEC.LV.ACB_10.ACBOCR_S'])

# 2. Visualization

|과거 상선|과거 색상|신규 상선|신규 색상|
|--|--|--|--|
|R|흑|L1|갈|
|S|적|L2|흑|
|T|청|L3|회|
|N|백/회|N|청|
|접지|녹|접지|녹 황 교차|

# 기본적인 계산 
1. 시간 대표값 (상위 시스템 표현 값 )

In [36]:
# 시간 대표값 

def calc_1h_mean(df):
    """
    상위 시스템에 올려주는 1시간 평균 값 계산 
    """

    df['one_hour'] = df['timestamp'].apply(lambda x : x.replace(minute=0)) # minute 이하 모두 0 

    mean_hour = df.groupby(['phase','one_hour',],as_index=False)['value'].mean().sort_values(by = ['phase', 'one_hour'])
    mean_hour.rename(columns={"value" : "mean_value_1h"}, inplace=True)

    df = pd.merge(df, mean_hour, on=['phase', 'one_hour'], how='left')
    df.drop(['one_hour'], axis=1, inplace=True)
    return df


# result = calc_1h_mean(result)

In [65]:


result['day'] = result['timestamp'].apply(lambda x : x.date())
result['time'] = result['timestamp'].apply(lambda x : x.time())
result['hour'] = result['timestamp'].apply(lambda x : x.replace(minute=0).time())
result['workday_name'] = result['timestamp'].apply(lambda x : x.day_name())
result['workday_number'] = result['timestamp'].apply(lambda x : x.weekday())
result['weeknumber'] = result['timestamp'].apply(lambda x : x.isocalendar()[1])

cols = ['value', 'minValue', 'maxValue']
for col in cols:
    result[col] = result[col].astype('float64') 

result

,phase,timestamp,value,minValue,maxValue,day,time,hour,workday_name,workday_number,weeknumber
0,A,2022-02-10 00:00:00,NaN,NaN,NaN,2022-02-10,00:00:00,00:00:00,Thursday,3,6
1,A,2022-02-10 00:01:00,0.000000,0.000000,NaN,2022-02-10,00:01:00,00:00:00,Thursday,3,6
2,A,2022-02-10 00:02:00,NaN,NaN,NaN,2022-02-10,00:02:00,00:00:00,Thursday,3,6
3,A,2022-02-10 00:03:00,0.000000,0.000000,NaN,2022-02-10,00:03:00,00:00:00,Thursday,3,6
4,A,2022-02-10 00:04:00,NaN,NaN,NaN,2022-02-10,00:04:00,00:00:00,Thursday,3,6
...,...,...,...,...,...,...,...,...,...,...,...
17275,C,2022-02-13 23:55:00,31.230059,31.230059,NaN,2022-02-13,23:55:00,23:00:00,Sunday,6,6
17276,C,2022-02-13 23:56:00,31.230059,31.230059,NaN,2022-02-13,23:56:00,23:00:00,Sunday,6,6
17277,C,2022-02-13 23:57:00,31.230059,31.230059,NaN,2022-02-13,23:57:00,23:00:00,Sunday,6,6
17278,C,2022-02-13 23:58:00,64.201591,64.201591,NaN,2022-02-13,23:58:00,23:00:00,Sunday,6,6


## 1. 기본 플롯

In [66]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_line_basic(df, phase):
    df_temp = df.loc[df["phase"] == phase].copy()
    # 1분 측정값
    fig = make_subplots(rows=1, cols=1, )
    fig.add_trace(go.Scatter(x=df_temp["timestamp"], y=df_temp["value"],
                        mode="lines", name="Values", line_color="#0066ff", fill=None, line=dict(dash="solid"),))            
    # # 1시간 평균값
    # fig.add_trace(go.Scatter(x=df_temp["timestamp"], y=df_temp["mean_value_1h"],
    #                     mode="lines", name="menaValue 1H", line_color="#6d4c41", fill=None, line=dict(dash="dot"),))
    # 1분 최소값
    fig.add_trace(go.Scatter(x=df_temp["timestamp"], y=df_temp["minValue"],
                        mode="lines", name="minValue", line_color="#33ccff", fill=None, line=dict(dash="dot"),))
    # 1분 최대값
    fig.add_trace(go.Scatter(x=df_temp["timestamp"], y=df_temp["maxValue"],
                        mode="lines", name="maxValue", line_color="#33ccff", fill="tonexty", line=dict(dash="dot"), fillcolor="rgba(204,238,255,0.5)",))

    fig.update_layout(showlegend=True, title_text=f"Date : {df_temp.day.min()}~{df_temp.day.max()}", 
                    yaxis = dict(title=f'Phase {phase} (%)'),
                    xaxis = dict(title='Time'),
                    autosize=False,
                    width=1200,height=400,)
    return fig

plot_line_basic(result, 'A')



## 2. THD(1분 평균값, High, Low) 연속 동일 값 나오는 부분 찾기 

[코드 처리 ]
1. 시간당 10% 이상 발생되는 곳 찾아 주기 -- 아마도 말단 측정 장비와 SCADA간 통신 지연(?)으로 데이터 Falling

In [54]:
date = "2022-02-12"
df_temp = result.copy()
df_temp['hour2'] = df_temp['timestamp'].apply(lambda x : x.replace(minute=0))
start_date = datetime.datetime.strptime(date, "%Y-%m-%d")
end_date = start_date - datetime.timedelta(days=2)
# mask = (df_temp.day == start_date.date()) | (df_temp.day == end_date.date())
mask = (df_temp.day == start_date.date())
df_temp = df_temp.loc[mask].reset_index(drop=True)
df_temp.hour = df_temp.hour.apply(lambda x : x.strftime("%H"))
# df_temp[['timestamp',	'value',	'minValue',	'maxValue',	'phase']][0:30]

In [55]:
date = "2022-02-12"
start_date = datetime.datetime.strptime(date, "%Y-%m-%d")
mask = (result.day == start_date.date())
df_temp = result.loc[mask].reset_index(drop=True)
df_temp[['timestamp',	'value',	'minValue',	'maxValue',	'phase']][0:30]

,timestamp,value,minValue,maxValue,phase
0,2022-02-12 00:00:00,0.0,0.0,NaN,A
1,2022-02-12 00:01:00,NaN,NaN,NaN,A
2,2022-02-12 00:02:00,NaN,NaN,NaN,A
3,2022-02-12 00:03:00,0.0,0.0,NaN,A
4,2022-02-12 00:04:00,NaN,NaN,NaN,A
5,2022-02-12 00:05:00,0.0,0.0,NaN,A
6,2022-02-12 00:06:00,0.0,0.0,NaN,A
7,2022-02-12 00:07:00,0.0,0.0,NaN,A
8,2022-02-12 00:08:00,0.0,0.0,NaN,A
9,2022-02-12 00:09:00,0.0,0.0,NaN,A


In [67]:
def calc_disconnet(result, date):
    df_temp = result.copy()
    start_date = datetime.datetime.strptime(date, "%Y-%m-%d")
    end_date = start_date - datetime.timedelta(days=7)
    mask = (df_temp.day >= end_date.date()) & (df_temp.day <= start_date.date())
    df_temp = df_temp.loc[mask]
    
    df_temp.hour = df_temp.hour.apply(lambda x : x.strftime("%H"))
    df_temp['hour2'] = df_temp['timestamp'].apply(lambda x : x.replace(minute=0))
    df_temp
    df_temp = df_temp[['day', 'hour2', 'hour', 'value', 'minValue', 'maxValue', 'phase']].loc[mask].reset_index(drop=True)
    
    days, hours, phases = df_temp.day.unique(), df_temp.hour.unique(), df_temp.phase.unique()

    ratio = []
    for order, info in enumerate(itertools.product(days, hours, phases)):
        mask = (df_temp['day'] == info[0]) & (df_temp['hour'] == info[1]) & (df_temp.phase == info[2])
        target = df_temp.loc[mask]
        
        time_info = target['hour2'].loc[mask][-1:].values[0]
        
        # for col in ['value', 'minValue', 'maxValue']:
        for col in ['value']:
            temp = target.loc[(target[col] >0) & (target[col] <100)].value.value_counts().to_frame()
            denominator = temp['value'].sum()
            value_once = round(temp.loc[temp['value'] == 1].sum()/denominator*100,2)
            value_twice = round(temp.loc[temp['value'] == 2].sum()/denominator*100,2)
            value_thirdtime = round(temp.loc[temp['value'] == 3].sum()/denominator*100,2)
            value_fourtimes = round(temp.loc[temp['value'] > 3].sum()/denominator*100,2)
        
            calc_ratio = [info[0], time_info, info[1], info[2], col, value_once.value, value_twice.value, value_thirdtime.value, value_fourtimes.value]
            ratio.append(calc_ratio)

    return pd.DataFrame(ratio, columns=['day', 'hour2', 'hour', 'phase', 'type',
                                        'once', 'twice', 'thirtime', 'over3']).fillna(0)
    

ratio=calc_disconnet(result, end_date)

In [68]:
temp = ratio.loc[ratio.phase == 'A']
x = [temp.day, temp.hour, temp.type]
fig = go.Figure()
fig.add_bar(x=x,y=temp.once,text = temp["once"], name='once', marker_color='#ad1457', )
fig.add_bar(x=x,y=temp.twice, text = temp["twice"],name='twice', marker_color='#e0e0e0', )
fig.add_bar(x=x,y=temp.thirtime, text = temp["thirtime"],name='third', marker_color ='#00695c', )
fig.add_bar(x=x,y=temp.over3, text = temp["over3"],name='thirdtime++', marker_color='#ff8a65', )
fig.update_layout(barmode='relative', title_text=f"phase 통신 주기, Period : {df_temp.day.min()} ~ {df_temp.day.max()}",
                    autosize=False, width=1600,height=400,)

fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1))
fig.show()

In [70]:

def plot_disconnet(ratio, phase):
    temp = ratio[['day','hour','phase','once','twice','thirtime','over3']].loc[ratio.phase == phase]
    x = [temp.day, temp.hour]
    fig = go.Figure()
    fig.add_bar(x=x,y=temp.once,text = temp["once"], name='once', marker_color='#ad1457', )
    fig.add_bar(x=x,y=temp.twice, text = temp["twice"],name='twice', marker_color='#e0e0e0', )
    fig.add_bar(x=x,y=temp.thirtime, text = temp["thirtime"],name='third', marker_color ='#00695c', )
    fig.add_bar(x=x,y=temp.over3, text = temp["over3"],name='thirdtime++', marker_color='#ff8a65', )
    fig.update_layout(barmode='relative', title_text=f"phase 통신 주기, Period : {df_temp.day.min()} ~ {df_temp.day.max()}",
                        autosize=False, width=1600,height=400,)
    fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1))
    
    return fig

plot_disconnet(ratio, 'B')

In [72]:

for order, phase in enumerate(['A','B','C'],1):
    tt = ratio[['day','hour','phase','once','twice','thirtime','over3']].loc[ratio.phase == phase]
    x = [tt.day, tt.hour]
    fig=go.Figure()
    fig.add_trace(go.Bar(x=x,y=tt.once,text = tt["once"], name='once', marker_color='#ad1457',), )
    fig.add_trace(go.Bar(x=x,y=tt.twice,text = tt["twice"], name='twice', marker_color='#e0e0e0',), )
    fig.add_trace(go.Bar(x=x,y=tt.thirtime,text = tt["thirtime"], name='third', marker_color='#00695c',), )
    fig.add_trace(go.Bar(x=x,y=tt.over3,text = tt["over3"], name='thirdtime++', marker_color='#ff8a65',), )
    
    fig.update_layout(barmode='relative', title_text=f"phase 통신 주기, Period : {df_temp.day.min()} ~ {df_temp.day.max()}",
                        autosize=False, width=1600,height=900,)
    fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1))
    fig.show()





In [ ]:
df = result.copy()
df_temp = pd.DataFrame(np.where(df[['value', 'minValue', 'maxValue']].diff() !=0, 0, 1), columns=['value_diff', 'minValue_diff', 'maxValue_diff'])
df_temp = pd.concat([df, df_temp], axis=1)


# df_temp = df_temp.groupby(['phase','day','hour'],as_index=False)[['value_diff','minValue_diff','maxValue_diff']].sum()

# if option == 'value':
#     fig = px.strip(df_temp, x="value_diff", y="hour", color='phase',facet_col="day")
# elif option == 'min':
#     fig = px.strip(df_temp, x="minValue_diff", y="hour", color='phase',facet_col="day")
# else:
#     fig = px.strip(df_temp, x="maxValue_diff", y="hour", color='phase',facet_col="day")
# fig.update_layout(showlegend=True, title_text=f"시간당 2분 이상 같은 값 연속 발생 누적 분 Name : {browsepath}, Date : {df_temp.day.min()}~{df_temp.day.max()}", 
#                 width=1200,height=600,)


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_strip_disconnet(df, option = 'value'):
    df_temp = pd.DataFrame(np.where(df[['value', 'minValue', 'maxValue']].diff() !=0, 0, 1), columns=['value_diff', 'minValue_diff', 'maxValue_diff'])
    df_temp = pd.concat([df, df_temp], axis=1)

    df_temp = df_temp.groupby(['phase','day','hour'],as_index=False)[['value_diff','minValue_diff','maxValue_diff']].sum()

    if option == 'value':
        fig = px.strip(df_temp, x="value_diff", y="hour", color='phase',facet_col="day")
    elif option == 'min':
        fig = px.strip(df_temp, x="minValue_diff", y="hour", color='phase',facet_col="day")
    else:
        fig = px.strip(df_temp, x="maxValue_diff", y="hour", color='phase',facet_col="day")
    fig.update_layout(showlegend=True, title_text=f"시간당 2분 이상 같은 값 연속 발생 누적 분 Name : {browsepath}, Date : {df_temp.day.min()}~{df_temp.day.max()}", 
                    width=1200,height=600,)

    return fig
    
plot_strip_disconnet(result)
    

In [ ]:
plot_strip_disconnet(result, 'min')

## 3. 다른주 동일요일 비교 -- 패턴 확인

In [ ]:
date = "2022-02-06"
df_temp = result.copy()
start_date = dt.datetime.strptime(date, "%Y-%m-%d")
end_date = start_date - dt.timedelta(days=7)
mask = (df_temp.day == start_date.date()) | (df_temp.day == end_date.date())
df_temp = df_temp.loc[mask].reset_index(drop=True)
df_temp

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


def plot_strip_thd_comp1(df, date, type = True):
    df_temp = df.copy()
    start_date = dt.datetime.strptime(date, "%Y-%m-%d")
    end_date = start_date - dt.timedelta(days=7)
    mask = (df_temp.day == start_date.date()) | (df_temp.day == end_date.date())
    df_temp = df_temp.loc[mask].reset_index(drop=True)

    if type:
    # Strip plot
        fig = px.strip(df_temp, x="value", y="day", color='phase')
    else:
        fig = px.strip(df_temp, x="value", y="hour", color='phase',facet_col="day")
        
    fig.update_layout(showlegend=True, title_text=f"{df_temp.weeknumber.unique()[0]}, {df_temp.weeknumber.unique()[1]} 작업주 -{df_temp.workday_name[-1:].values[0]} THD 경향   Name : {browsepath}, Date : {date}", 
                        width=1200,height=600, )
    fig.update_xaxes(matches='x')
    fig.add_vrect(x0=20, x1=100, 
              annotation_text="abnormal", annotation_position="top",
              fillcolor="red", opacity=0.1, line_width=0)
    
    return fig
    
date = "2022-02-06"
plot_strip_thd_comp1(result, date)



In [ ]:
plot_strip_thd_comp1(result, date, False)

### 어제와 비교 

In [ ]:
def plot_strip_thd_comp2(df, date, type = True):

    df_temp = df.copy()
    start_date = dt.datetime.strptime(date, "%Y-%m-%d")
    end_date = start_date - dt.timedelta(days=1)
    mask = (df_temp.day == start_date.date()) | (df_temp.day == end_date.date())
    df_temp = df_temp.loc[mask].reset_index(drop=True)

    if type:
        # Strip plot
        fig = px.strip(df_temp, x="value", y="day", color='phase')
    else:
        fig = px.strip(df_temp, x="value", y="hour", color='phase',facet_col="day")
        
    fig.update_layout(showlegend=True, title_text=f"{df_temp.workday_name.unique()[0]}, {df_temp.workday_name.unique()[1]} 작업주 -{df_temp.workday_name[-1:].values[0]} THD 경향   Name : {browsepath}, Date : {date}", 
            width=1200,height=600,)
    fig.update_xaxes(matches='x')
    fig.add_vrect(x0=20, x1=100, 
              annotation_text="abnormal", annotation_position="top",
              fillcolor="red", opacity=0.1, line_width=0)
    return fig

date = "2022-02-06"
plot_strip_thd_comp2(result, date)

In [ ]:
date = "2022-02-06"
plot_strip_thd_comp2(result, date, False)

## 상별 상관관계 

In [ ]:
from functools import reduce

def corr_matrix(df, date):
    
    df_temp = df.copy()
    start_date = dt.datetime.strptime(date, "%Y-%m-%d")
    df_temp = df.loc[df.day == start_date.date()]
    df_temp = df_temp.drop(['time',	'hour',	'workday_name',	'workday_number',	'weeknumber'], axis=1)

    temp = pd.DataFrame()
    phaseA = df_temp.loc[df_temp.phase == "A"].copy()
    phaseA.rename(columns = {'value' :"value_A",	'minValue' :"minValue_A",	'maxValue' :"maxValue_A",	'mean_value_1h' :"mean_value_1h_A"}, inplace=True)
    phaseB = df_temp.loc[df_temp.phase == "B"].copy()
    phaseB.rename(columns = {'value' :"value_B",	'minValue' :"minValue_B",	'maxValue' :"maxValue_B",	'mean_value_1h' :"mean_value_1h_B"}, inplace=True)
    phaseC = df_temp.loc[df_temp.phase == "C"].copy()
    phaseC.rename(columns = {'value' :"value_C",	'minValue' :"minValue_C",	'maxValue' :"maxValue_C",	'mean_value_1h' :"mean_value_1h_C"}, inplace=True)
    temp = reduce(lambda left, right: pd.merge(left, right, on=['timestamp','day']),[phaseA, phaseB, phaseC],)

    temp = temp.drop(['timestamp', 'day','phase_x','phase_y','phase'], axis=1).astype('float64').corr().fillna(0)

    fig = px.imshow(temp, text_auto=True, aspect="auto")
    fig.update_layout(showlegend=True, title_text=f"A,B,C상 상관관계  Name : {browsepath}, Date : {date}", 
                    width=900,height=600,)
    fig.update_xaxes(side="top")
    return fig

date = "2022-02-06"
corr_matrix(result, date)

## 상별 상관관계 플랏 
#### 4. 1일 시간별 상관 관계 

In [ ]:
from plotly.subplots import make_subplots
COLORS = px.colors.qualitative.Dark24


In [ ]:
FIGURE_RANGE = 100      # 그림 표현 범위 
THD_ABNORMAL_THRESHOLD = 15 # THD 이상치 범위 

def plot_scatter_day(df, date): 

    df_temp = df.copy()

    start_date = dt.datetime.strptime(date, "%Y-%m-%d")
    df_temp = df_temp.loc[df_temp.day == start_date.date()]

    phases = ['AB', 'AC', 'BC']
    hours = df_temp.hour.unique()

    fig = make_subplots(rows=1, cols=3, )  #subplot_titles=("Phase A", "Phase B", "Phase C")
    for col_order, phase in enumerate(phases, 1):
        for hour_order, hour in enumerate(hours):
            mask = (df_temp.hour == hour)
            temp = df_temp.loc[mask].reset_index(drop=True)
            pahse_A_byhour = temp.loc[temp.phase== phase[0]]
            pahse_B_byhour = temp.loc[temp.phase== phase[1]]

            fig.add_trace(go.Scatter(x=pahse_A_byhour["value"], y=pahse_B_byhour["value"],
                                    mode="markers", name=temp.hour[0].strftime("%H"), text=temp.time,  marker_color=COLORS[hour_order], showlegend = False,), row=1, col=col_order, )
            fig.add_shape(type="line", x0=THD_ABNORMAL_THRESHOLD, y0=0, x1=THD_ABNORMAL_THRESHOLD, y1=FIGURE_RANGE,line=dict(color="#757575", dash="dot", ),row=1, col=col_order,)
            fig.add_shape(type="line", x0=0, y0=THD_ABNORMAL_THRESHOLD, x1=FIGURE_RANGE, y1=THD_ABNORMAL_THRESHOLD,line=dict(color="#757575", dash="dot", ),row=1, col=col_order,)
            fig.add_shape(type="line", x0=0, y0=0, x1=FIGURE_RANGE, y1=FIGURE_RANGE,line=dict(color="#757575", dash="dot", ),row=1, col=col_order,)

    fig.update_layout(showlegend=True, title_text=f"1일 시간별 A,B,C THD 상관관계 <br> Name : {browsepath}, Date : {df_temp.day.min()}, THD Threshold : {THD_ABNORMAL_THRESHOLD}", 
                    xaxis1 = dict(title='Phase A (%)', range=[0, FIGURE_RANGE]),
                    yaxis1 = dict(title='Phase B (%)', range=[0, FIGURE_RANGE]),
                    xaxis2 = dict(title='Phase A (%)', range=[0, FIGURE_RANGE]),
                    yaxis2 = dict(title='Phase C (%)', range=[0, FIGURE_RANGE]),
                    xaxis3 = dict(title='Phase B (%)', range=[0, FIGURE_RANGE]),
                    yaxis3 = dict(title='Phase C (%)', range=[0, FIGURE_RANGE]),
                    autosize=False,
                    width=1600,height=800,)
    return fig

date = '2022-02-13'
plot_scatter_day(result, date)

#### 5. 1 주일 시간별 상관 관계 

In [ ]:
result

In [ ]:
COLORS = px.colors.qualitative.Dark24
FIGURE_RANGE = 100      # 그림 표현 범위 
def plot_scatter_week(df, date): 
    df_temp = df.copy()

    start_date = dt.datetime.strptime(date, "%Y-%m-%d")
    end_date = start_date - dt.timedelta(days=7)
    mask = (df_temp.day <= start_date.date()) & (df_temp.day >= end_date.date())
    df_temp = df_temp.loc[mask].reset_index(drop=True)

    phases = ['AB', 'AC', 'BC']
    days = df_temp.day.unique()
    fig = make_subplots(rows=1, cols=3, )  #subplot_titles=("Phase A", "Phase B", "Phase C")
    for col_order, phase in enumerate(phases, 1):
        for day_order, day in enumerate(days):
            mask = (df_temp.day == day)
            temp = df_temp.loc[mask]
            pahse_A_byweek = temp.loc[temp.phase== phase[0]].reset_index(drop=True)
            pahse_B_byweek = temp.loc[temp.phase== phase[1]].reset_index(drop=True)
            
            fig.add_trace(go.Scatter(x=pahse_A_byweek["value"], y=pahse_B_byweek["value"],
                                    mode="markers", name=pahse_A_byweek.workday_name[0], text=[pahse_A_byweek.workday_name[0]],
                                    marker_color=COLORS[day_order], showlegend = False,), row=1, col=col_order, )
            fig.add_shape(type="line", x0=THD_ABNORMAL_THRESHOLD, y0=0, x1=THD_ABNORMAL_THRESHOLD, y1=FIGURE_RANGE,line=dict(color="#757575", dash="dot", ),row=1, col=col_order,)
            fig.add_shape(type="line", x0=0, y0=THD_ABNORMAL_THRESHOLD, x1=FIGURE_RANGE, y1=THD_ABNORMAL_THRESHOLD,line=dict(color="#757575", dash="dot", ),row=1, col=col_order,)
            fig.add_shape(type="line", x0=0, y0=0, x1=FIGURE_RANGE, y1=FIGURE_RANGE,line=dict(color="#757575", dash="dot", ),row=1, col=col_order,)

    fig.update_layout(showlegend=True, title_text=f"1주일 A,B,C THD 상관관계 <br> Name : {browsepath}, Date : {df_temp.day.min()}~{df_temp.day.max()}, THD Threshold : {THD_ABNORMAL_THRESHOLD}", 
                    xaxis1 = dict(title='Phase A (%)', range=[0, FIGURE_RANGE]),
                    yaxis1 = dict(title='Phase B (%)', range=[0, FIGURE_RANGE]),
                    xaxis2 = dict(title='Phase A (%)', range=[0, FIGURE_RANGE]),
                    yaxis2 = dict(title='Phase C (%)', range=[0, FIGURE_RANGE]),
                    xaxis3 = dict(title='Phase B (%)', range=[0, FIGURE_RANGE]),
                    yaxis3 = dict(title='Phase C (%)', range=[0, FIGURE_RANGE]),
                    autosize=False,
                    width=1600,height=800,)

    return fig 

date = '2022-02-13'
plot_scatter_week(result, date)

### 6. 1일 시간대별 THD 발생 경향 

In [ ]:
def plot_scatter_time_thd_day(df, date, option = True):

    df_temp = df.copy()    
    start_date = dt.datetime.strptime(date, "%Y-%m-%d")
    df_temp = df_temp.loc[df_temp.day == start_date.date()]
    
    if option: 
        fig = px.strip(df_temp, x="value", y="workday_name", color='phase')

    else: 
        fig = px.strip(df_temp, x="value", y="hour", color='phase',facet_col="workday_name")

    fig.update_layout(showlegend=True, title_text=f"1일 시간대별 THD 발생 경향 Date : {df_temp.day.min()}", 
                    width=1500,height=600,)
    return fig 

date = '2022-02-13'
plot_scatter_time_thd_day(result, date)

In [ ]:
plot_scatter_time_thd_day(result, date, False)

### 7. 1주일 시간대별 THD 발생 경향 

In [ ]:
def plot_scatter_time_thd_week(df, date,  option = True):

    df_temp = df.copy()
    start_date = dt.datetime.strptime(date, "%Y-%m-%d")
    end_date = start_date - dt.timedelta(days=3)
    mask = (df_temp.day <= start_date.date()) & (df_temp.day >= end_date.date())
    df_temp = df_temp.loc[mask].reset_index(drop=True)

    if option:
        fig = px.strip(df_temp, x="value", y="workday_name", color='phase')

    else:
        fig = px.strip(df_temp, x="value", y="hour", color='phase',facet_col="workday_name")

    fig.update_layout(showlegend=True, title_text=f"작업주 THD 경향   NDate : {df_temp.day.min()}~{df_temp.day.max()}", 
                width=1500,height=600,)
    return fig

date = '2022-02-13'
plot_scatter_time_thd_week(result, date)


In [ ]:
plot_scatter_time_thd_week(result, date,  option = False)

In [ ]:
df_temp

In [ ]:
fig_list = [
    "fig1. 기간 THD 정보",
    "fig2. 통신 누락 분포",
    "fig3. 1주일 간격 THD 비교",
    "fig4. 1일 간격 THD 비교",
    "fig5. 상별 상관관계",
    "fig6. 최근 2일 상별 상관관계",
    "fig7. 최근 1주일 상별 상관관계",
    "fig8. 1일 시간대별 THD 발생 경향",
    "fig9. 최근 1주일 일별 THD 발생 경향",
]

if any(format  in "fig4. 1일 간격 THD 비교" for format  in fig_list):
    print(True)
else:
    print(False)



In [ ]:
fig_list